# Is it going to rain tomorrow?

This project is for me to practice my machine learning chops while I'm in the early parts of my class. I'll be exploring the [Rain in Australia](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package) Kaggle dataset and attempt to build a model to predict whether or not it will rain the next day for a given observation. Let's see how it goes!

In [ ]:
threads = -1

In [ ]:
import dill
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from functools import partial
from missingpy import MissForest
from scipy import stats
from tqdm.notebook import tqdm

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, cross_val_predict, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

## Exploring the data

First, we need to actually download the data. Thankfully Kaggle provides a helpful [API](https://github.com/Kaggle/kaggle-api). This will be installed as part of the `requirements.txt`.

In [ ]:
%%bash

if ! [ -d "./data" ]; then
    mkdir ./data && cd ./data
    kaggle datasets download jsphyg/weather-dataset-rattle-package --unzip
fi

In [ ]:
data = pd.read_csv("./data/weatherAUS.csv")
data.head()

In [ ]:
data.info()

Looks like `Evaporation`, `Sunshine`, `Cloud9am`, and `Cloud3pm` all have a lot of missing observations. I'm inclined to remove them since that's a decent chunk of data with some missing observations. But before I do that, let's just take a quick peak at how it relates to `RainTomorrow` just to see if there's any interesting relationship there.

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
sns.boxplot(x="RainTomorrow", y="Evaporation", data=data, ax=axs[0][0])
sns.boxplot(x="RainTomorrow", y="Sunshine", data=data, ax=axs[0][1])
sns.boxplot(x="RainTomorrow", y="Cloud9am", data=data, ax=axs[1][0])
sns.boxplot(x="RainTomorrow", y="Cloud3pm", data=data, ax=axs[1][1])
plt.show()

It looks like all of them seem to have a big impact on `RainTomorrow`, though it's hard to tell with `Evaporation` because of the wonky distribution. Let's do a test to compare the means of both groups to see. None of these variables look normal though, so we probably can't do a t-test, but let's take a closer look at the distribution of one case for fun.

In [ ]:
sns.distplot(data["Evaporation"][data["RainTomorrow"] == "Yes"])
plt.show()

That doesn't look quite normal to me, to further reinforce, we'll do a [D'Agostino-Pearson](https://en.wikipedia.org/wiki/D%27Agostino%27s_K-squared_test) test for normality.

In [ ]:
stats.normaltest(data["Evaporation"][data["RainTomorrow"] == "No"].dropna())

Definitely not normal, as expected. So we can't do a t-test, but a common non-parametric alternative is the [Mann-Whitney U test](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test). So let's try that!

I was only interested in `Evaporation`, but might as well do all of them.

In [ ]:
rain_yes = data[data["RainTomorrow"] == "Yes"]
rain_no = data[data["RainTomorrow"] == "No"]

def rain_tomorrow_mannwhitneyu_test(variable):
    return stats.mannwhitneyu(rain_yes[variable].dropna(), rain_no[variable].dropna())

print(f"""
Evaporation: {rain_tomorrow_mannwhitneyu_test('Evaporation')}
Sunshine: {rain_tomorrow_mannwhitneyu_test('Sunshine')}
Cloud9am: {rain_tomorrow_mannwhitneyu_test('Cloud9am')}
Cloud3pm: {rain_tomorrow_mannwhitneyu_test('Cloud3pm')}
""")

Wow, those are some significant p-values. Just like I expected. Looks like `Evaporation` is also highly significant.

Since they are so important to determining `RainTomorrow`, I want to keep them in. During preparation, I can try imputing as well as dropping `NaN` rows.

In [ ]:
data.describe()

In [ ]:
data.hist(figsize=(15,15))
plt.show()

In [ ]:
%%time

!mkdir -p ./images

pair_plot = sns.pairplot(data, hue="RainTomorrow")
pair_plot.savefig("./images/pairplot.png")

Although I can pick out some patterns from these scatter plots, it seems like `Evaporation` is the most powerful delineator in terms of `RainTomorrow`. I'll want to keep that in mind moving forward.

## Preparing the data

Ok, at this point I think I've done enough initial exploration and engineering. Let's go ahead and start preparing the data for machine learning and see how well it can perform.

Let's begin with a little feature engineering. One thing to note in particular is how a lot of these variables have a morning and evening variant. Let's make a new column for each of these to represent the change in said variable.

Another interesting feature could be how much `Evaporation` there was for how much `Sunshine` there was in a day. My intuition tells me that if there was a lot of `Evaporation` and not a lot of `Sunshine` (it was already a cloudy day), the chance of it raining would be higher.

Also if there was a lot of `Rainfall` and little `Sunshine`, it should continue to rain.

In [ ]:
def engineer_features(columns, X):
    df = pd.DataFrame(X, columns=columns)
    
    df["ChangeInCloud"] = df["Cloud3pm"] - df["Cloud9am"]
    df["ChangeInHumidity"] = df["Humidity3pm"] - df["Humidity9am"]
    df["RangeOfTemp"] = df["MaxTemp"] - df["MinTemp"]
    df["ChangeInPressure"] = df["Pressure3pm"] - df["Pressure9am"]
    df["ChangeInTemp"] = df["Temp3pm"] - df["Temp9am"]
    df["ChangeInWindSpeed"] = df["WindSpeed3pm"] - df["WindSpeed9am"]
    
    df["EvaporationPerSunshine"] = df["Evaporation"] / df["Sunshine"]
    df["RainfallPerSunshine"] = df["Rainfall"] / df["Sunshine"]
    
    return df

Now I'll set up a pipeline for transforming the categorical and numeric variables accordingly.

It's worth noting this will drop `RISK_MM`. Discussion [here](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package/discussion/78316) regarding `RISK_MM` as it involves data leakage.

For `Date`, I want to split it into month and day for two _"new"_ features.

In [ ]:
def pipeline(estimator=None, imputer=(SimpleImputer(strategy="most_frequent"), SimpleImputer()), engineer=True):
    categorical_columns = ["Location", "WindGustDir", "WindDir9am", "WindDir3pm", "RainToday"]
    numeric_columns = ["MinTemp", "MaxTemp", "Rainfall", "Evaporation", "Sunshine", "WindGustSpeed", "WindSpeed9am", "WindSpeed3pm",
                       "Humidity9am", "Humidity3pm", "Pressure9am", "Pressure3pm", "Cloud9am", "Cloud3pm", "Temp9am", "Temp3pm"]

    date_transformer = Pipeline([
        ("split", FunctionTransformer(lambda s: s.str.split('-', expand=True).iloc[:, 1:])),
        ("imputer", imputer[0])
    ])
    
    categorical_transformer = Pipeline([
        ("imputer", imputer[0]),
        ("one_hot", OneHotEncoder(sparse=False))
    ])
    
    numeric_transformer = Pipeline(list(filter(None, [
        ("imputer", imputer[1]),
        ("add_one", FunctionTransformer(lambda X: X + 1)),
        ("engineer", FunctionTransformer(partial(engineer_features, numeric_columns))) if engineer else None,
        ("scaler", StandardScaler())
    ])))

    column_transformer = ColumnTransformer([
        ("date", date_transformer, "Date"),
        ("categorical", categorical_transformer, categorical_columns),
        ("numeric", numeric_transformer, numeric_columns)
    ])
    
    return Pipeline(list(filter(None, [
        ("preprocessor", column_transformer),
        ("estimator", estimator) if estimator is not None else None
    ])))

## Model selection

Great, I've set up our data preprocessing pipeline, now it's time to find a good model!

To begin, I'll need to split our data into a train and test set and preprocess them.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, data["RainTomorrow"], test_size=0.2, stratify=data["RainTomorrow"])

In [ ]:
y_preprocessor = LabelEncoder()
y_train_p = y_preprocessor.fit_transform(y_train)

Ok, I'll start by trying several different models on just the default parameters. Then I'll pick one to do some more in-depth training with.

In [ ]:
models = [
    RandomForestClassifier(),
    Perceptron(),
    SGDClassifier(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
]

results = pd.DataFrame(columns=["Model", "Cross Validation Mean"])
for model in tqdm(models):
    avg_score = np.mean(cross_val_score(pipeline(model), X_train, y_train_p, n_jobs=threads, verbose=1))
    results.loc[len(results)] = [model.__class__.__name__, avg_score]

In [ ]:
results.sort_values(by="Cross Validation Mean", ascending=False)

In [ ]:
sns.barplot(x="Cross Validation Mean", y="Model", data=results.sort_values(by="Cross Validation Mean", ascending=False))
plt.show()

Looks like I'll go with random forest since it has high accuracy and it's a model I'm familiar with.

## Fine-tuning

One of the first things I want to examine is the confusion matrix for these predictions. In particular with predicting rainfall, we may be more interested in recall, especially since on average it's more likely to not rain than to do rain, so the model can just lean on predicting _No_ and be highly accurate; but failing to predict rain can have more consequences.

In [ ]:
clf = pipeline(RandomForestClassifier())
y_pred = cross_val_predict(clf, X_train, y_train_p, cv=10, n_jobs=threads, verbose=1)

In [ ]:
print(list(zip(y_preprocessor.transform(y_preprocessor.classes_), y_preprocessor.classes_)))
sns.heatmap(confusion_matrix(y_train_p, y_pred), annot=True, fmt="d")
plt.show()

In [ ]:
print(classification_report(y_train_p, y_pred))

I was worried this would happen. The recall on the _Yes_ class where it does rain is only $50\%$, that's horrible. I'll try re-preprocessing the data without features and imputation to see if how those had an effect.

First, without imputing.

In [ ]:
dropped = data.dropna()
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(dropped, dropped["RainTomorrow"], test_size=0.2, stratify=dropped["RainTomorrow"])
y_preprocessor_d = LabelEncoder()
y_train_d_p = y_preprocessor_d.fit_transform(y_train_d)
print("Number of observations:", X_train_d.shape[0])


clf_no_impute = pipeline(RandomForestClassifier())
y_pred_no_impute = cross_val_predict(clf_no_impute, X_train_d, y_train_d_p, cv=10, n_jobs=threads, verbose=1)
print(classification_report(y_train_d_p, y_pred_no_impute))

Now without engineering.

In [ ]:
clf_no_engineer = pipeline(RandomForestClassifier(), engineer=False)
y_pred_no_engineer = cross_val_predict(clf_no_engineer, X_train, y_train_p, cv=10, n_jobs=threads, verbose=1)
print(classification_report(y_train_p, y_pred_no_engineer))

Now without either.

In [ ]:
clf_neither = pipeline(RandomForestClassifier(), engineer=False)
y_pred_neither = cross_val_predict(clf_neither, X_train_d, y_train_d_p, cv=10, n_jobs=threads, verbose=1)
print(classification_report(y_train_d_p, y_pred_neither))

Looks like none of that really mattered, though removing imputation seemed to improve general performance slightly, but at the cost of a significant reduction in observations. I can try using a random forest to impute the data, however it is very slow. I'll see how well it works before committing to it. Hopefully, being able to keep a lot of data as well as better imputation will improve performance.

In [ ]:
clf_test_rf = pipeline(RandomForestClassifier(), imputer=(SimpleImputer(strategy="most_frequent"), MissForest(n_jobs=threads)))
y_pred_test_rf = cross_val_predict(clf_test_rf, X_train, y_train_p, cv=2, verbose=1)

In [ ]:
print(classification_report(y_train_p, y_pred_test_rf))

Well that didn't show any improvement at all over just the `SimpleImputer`. In fact, it seems to perform worse! I guess I won't bother with that. It's clear that imputation isn't helping or hindering here, so instead I should focus on hyperparameter optimization to improve recall.

### Parameter search

Ok, now I'll run my model through a randomized parameter search and optimize for recall instead of accuracy. Of course this is based on my own assumption. If this were a real task, there would be a lot of work to determine what the problem we are trying to solve is to determine the correct metric to use.

In [ ]:
param_distributions = {
    "estimator__n_estimators" : [10, 50, 100, 300, 500, 1000],
    "estimator__max_depth": [3, 5, 10, 20, 30, 40, 50],
    "estimator__min_samples_split": [2, 5, 10, 15, 30],
    "estimator__min_samples_leaf": [1, 3, 5, 10, 15, 20],
    "estimator__max_features": ["auto", "sqrt", "log2"]
}

clf_rs = RandomizedSearchCV(pipeline(RandomForestClassifier()), param_distributions, n_iter=100, scoring="recall", cv=10, n_jobs=threads, verbose=1)
clf_rs = clf_rs.fit(X_train, y_train_p)

In [ ]:
clf_rs.cv_results_

In [ ]:
dill.dump_session("rain.db")

## Conclusions

## What I've learned

* Perform a train/test split before doing EDA.
* Perform more EDA to gain more insight into relationships between variables.
* Make options for pipeline parameters so they can be included as part of the parameter search.
* Consider a different option for comparing multiple models.
* Consider an ensemble method like a voting classifier.